[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/cli/singlepoint.ipynb)

# Single Point

`janus-core` contains various machine learnt interatomic potentials (MLIPs), including MACE based models (MACE-MP, MACE-OFF), CHGNet, Orb and more, full list on https://github.com/stfc/janus-core.

Other will be added as their utility is proven beyond a specific material.

## Set up environment (optional)

These steps are required to run this tutorial with Google Colab. To do so, uncomment and run the cell below.

This will replace pre-installed versions of `numpy` and `torch` in Colab with versions that are known to be compatible with `janus-core`.

It may be possible to skip the steps that uninstall and reinstall `torch`, which will save a considerable amount of time.

These instructions but may work for other systems too, but it is typically preferable to prepare a virtual environment separately before running this notebook if possible.

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall numpy -y # Uninstall pre-installed numpy

# ! pip uninstall torch torchaudio torchvision transformers -y # Uninstall pre-installed torch
# ! uv pip install torch==2.5.1 # Install pinned version of torch

# ! uv pip install janus-core[mace,orb,visualise] data-tutorials --system # Install janus-core with MACE, Orb, and WeasWidget, and data-tutorials

# get_ipython().kernel.do_shutdown(restart=True) # Restart kernel to update libraries. This may warn that your session has crashed.

To ensure you have the latest version of `janus-core` installed, compare the output of the following cell to the latest version available at https://pypi.org/project/janus-core/

In [ ]:
from janus_core import __version__

print(__version__)

## Prepare data

Use `data_tutorials` to get the data required for this tutorial:

In [ ]:
from data_tutorials.data import get_data

get_data(
    url="https://raw.githubusercontent.com/stfc/janus-core/main/docs/source/tutorials/data/",
    filename=["NaCl-set.xyz"],
    folder="../data",
)

## Command-line help and options

Once `janus-core` is installed, the `janus` CLI command should be available:

In [ ]:
! janus --help

Help for individual `janus` commands also be requested, describing all available options: 

In [ ]:
! janus singlepoint --help

## Running single point calculations

First, we need a file to perform calculations on. Here, we build a periodic salt structure, visualise it, and write out to a file.

<div class="alert alert-block alert-info">
<b>Tip:</b> We use the ASE and WEAS Widget libaries here to build the structure and visualise it.
We discuss these tools in more detail in the Python tutorials. 
</div>

In [ ]:
from ase.build import bulk
from ase.io import write
from weas_widget import WeasWidget

NaCl = bulk("NaCl", "rocksalt", a=5.63, cubic=True)

write("../data/NaCl.xyz", NaCl)

v=WeasWidget()
v.from_ase(NaCl)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

Now we can use an MLIP to run single point calculations. By default, this uses the MACE-MP model:

In [ ]:
! janus singlepoint --arch mace_mp --struct ../data/NaCl.xyz --no-tracker

Results from CLI calculations are saved in a newly created directory, `janus_results`:

In [ ]:
! ls janus_results/

We can now read the output file back into ASE, to see the saved energy, stresses, and forces:

In [ ]:
from ase.io import read

results = read("janus_results/NaCl-results.extxyz")

In [ ]:
print(f"Energy: {results.info['mace_mp_energy']}")
print()
print(f"Stress: {results.info['mace_mp_stress']}")
print()
print(f"Forces: {results.arrays['mace_mp_forces']}")

We can check the units corresponding to these quantities, which are also saved in the "info" dictionary:

In [ ]:
print(results.info["units"])

## Using configuration files

All options to `janus` commands can be specified through a YAML input file. These are text files of the form:

```bash
key: value
list_key:
  - list_value_1
  - list_value_2
nested_key_1:
  nested_key_2: nested_value
```

which, in Python, would correspond to a dictionary of the form:

```python
{
    "key": value,
    "list_key": [list_value_1, list_value_2],
    "nested_key": {"nested_key_2": nested_value},
}
```

Although you can specify every option, let's first write a minimal configuration file for `janus singlepoint`:

In [ ]:
%%writefile singlepoint_config_1.yml

arch: mace_mp
struct: ../data/NaCl.xyz
tracker: False

In [ ]:
! cat singlepoint_config_1.yml

We can then use this configuration file to re-run the calculation:

In [ ]:
! janus singlepoint --config singlepoint_config_1.yml

Options in the configuration files can be overwritten when running a command. For example, the following configuration file also defines the `file_prefix`, which specifies any directories and the prefix of the file names to be output:

In [ ]:
%%writefile singlepoint_config_2.yml

arch: mace_mp
struct: ../data/NaCl.xyz
file_prefix: examples/NaCl
tracker: False

We can now use this configuration file, but replace `file_prefix`:

In [ ]:
! janus singlepoint --config singlepoint_config_2.yml --file-prefix outputs/salt

<div class="alert alert-block alert-info">
<b>Note:</b> In the CLI, multi-word arguments must be separated by "-".
In configuration files, they are preferably separated by "_", although "-" should be converted automatically.
</div>

This creates a new output directory, `outputs`, as well as starting all output files with "salt-"

In [ ]:
! ls outputs

## Running multiple calculations

By default, results from `janus singlepoint` are saved in two forms. For energies, this is an "info" dictionary, and a "results" dictionary.

One set of dictionary keys will be labelled with the MLIP model used ("mace_mp_energy", "mace_mp_stress", "mace_mp_forces"), which ensures these are not overwritten when running calculations with multiple models.

The unlabelled set of keys ("energy", "forces", "stress") allow ASE to use the results for further calculations, but will be overwritten if a new calculation is run.

In [ ]:
print(results.info.keys())
print(results.arrays.keys())
print(results.calc.results.keys())

print()
print(results.calc.results["energy"])
print(results.get_potential_energy())
print(results.get_total_energy())

For example, if we re-run the calculation using Orb, saving the results to a new output file, we see that "mace_mp_energy" is still saved, other unlabelled results are updated:

In [ ]:
! janus singlepoint --struct janus_results/NaCl-results.extxyz --arch orb --out janus_results/NaCl-updated-results.extxyz --no-tracker

In [ ]:
updated_results = read("janus_results/NaCl-updated-results.extxyz")

In [ ]:
print(updated_results.info.keys())
print(updated_results.arrays.keys())
print(updated_results.calc.results.keys())

print()
print(updated_results.info["mace_mp_energy"])
print(updated_results.info["orb_energy"])
print(updated_results.calc.results["energy"])
print(updated_results.get_potential_energy())
print(updated_results.get_total_energy())

## Output files

In addition to the resulting structure file, all calculations run through the CLI generate log (`NaCl-singlepoint-log.yml`) and summary (`NaCl-singlepoint-summary.yml`) files.

In this case, the log file captures timestamps for the start and end of the calculation, but it will also capture any Python warnings generated, and carbon tracking information:

In [ ]:
! cat janus_results/NaCl-singlepoint-log.yml

The summary file contains:

- The main command run
- The CLI options specified
- Basic information about the structure
- Output files generated by the calculation
- Start and end times of the calculation
- Carbon tracking summary, if applicable 

In [ ]:
! cat janus_results/NaCl-singlepoint-summary.yml

## Reusing configuration files

We can extract the configuration used to run our first calculation from the summary file, reusing it with a slight modification.

First, we read in the configuration via the summary file:

In [ ]:
import yaml

with open("janus_results/NaCl-singlepoint-summary.yml", encoding="utf8") as file:
    summary = yaml.safe_load(file)

config = summary["config"]
print(config)

Next, let's change the structure to run the calculations on, ensure we read all images the file (this is actually the default for single point calculations), and change the calculated property:

In [ ]:
config["struct"] = "../data/NaCl-set.xyz"
config["read_kwargs"] = {"index": ":"} # Key word arguments, passed to ase.io.read`
config["properties"] = ["hessian"] # This must be a list, even for a single quantity

Now, we write this file out, and use it to run our calculation:

In [ ]:
with open("singlepoint_config_3.yml", "w", encoding="utf8") as file:
    yaml.dump(config, file)

In [ ]:
! cat singlepoint_config_3.yml

In [ ]:
! janus singlepoint --config singlepoint_config_3.yml

We can read in these results, ensuring we read in all structures from the file:

In [ ]:
NaCl_set_results = read("janus_results/NaCl-set-results.extxyz", index=":")

In [ ]:
print(NaCl_set_results[0].info["mace_mp_hessian"])

## Setting keyword arguments

All `janus` commands accept keyword arguments (kwargs), which are input in the form of Python dictionaries: `{"key": value}`.

One useful example is passing options that are specific to an MLIP calculator.

For example, all MLIPs have an option to run with D3 dispersion correction, through the `dispersion` option. In the command-line, this can be set using:

In [ ]:
! janus singlepoint --config singlepoint_config_1.yml --calc-kwargs "{'dispersion': True}" --out janus_results/NaCl-dispersion-results.extxyz

To avoid the complications of nested quotations, setting this in the configuration file is preferable:

In [ ]:
%%writefile singlepoint_config_4.yml

arch: mace_mp
struct: ../data/NaCl.xyz
tracker: False
calc_kwargs:
  dispersion: True
out: janus_results/NaCl-dispersion-results.extxyz

In [ ]:
! janus singlepoint --config singlepoint_config_4.yml

Comparing the results before and after the correction:

In [ ]:
dispersion_results = read("janus_results/NaCl-dispersion-results.extxyz")

print(f"Original results: {results.info['mace_mp_energy']}")
print(f"Results with dispersion correction: {dispersion_results.info['mace_mp_d3_energy']}")